# 基站覆盖问题

本笔记本旨在使用 OptVerse 来复现 [Gurobi 基站覆盖问题示例](https://github.com/Gurobi/modeling-examples/blob/master/cell_tower_coverage/cell_tower.ipynb)。
此示例演示了如何解决基站选址问题，以为尽可能多的人群提供信号覆盖。

包含本示例和其他示例的代码库可通过 [CodeHub](link) 获取

## 问题描述

以下部分取自 [Gurobi 基站覆盖问题示例](https://github.com/Gurobi/modeling-examples/blob/master/cell_tower_coverage/cell_tower.ipynb)。

一家电信公司需要建设一组基站来为某个城市的居民提供信号覆盖。已经确定了一些可以建设基站的潜在地点。基站具有固定的覆盖范围，由于预算限制，只能建设有限数量的基站。在这些限制条件下，公司希望为尽可能多的人口提供覆盖。为了简化问题，公司将要覆盖的区域划分为一组区域，每个区域都有已知的人口数量。目标是选择在哪些潜在地点建设基站，以覆盖尽可能多的人口。

基站覆盖问题是最大覆盖选址问题的一个实例。它也与集合覆盖问题相关。

## 求解方法

以下部分取自 [Gurobi 基站覆盖问题示例](https://github.com/Gurobi/modeling-examples/blob/master/cell_tower_coverage/cell_tower.ipynb)。

数学规划是一种声明式方法，建模者构建一个数学优化模型来捕捉复杂决策问题的关键要素。OptVerse 优化器使用最先进的数学和计算机科学技术来求解此类模型。

一个数学优化模型包含五个组成部分：

* 集合和下标
* 参数
* 决策变量
* 目标函数
* 约束条件

接下来我们为基站覆盖问题提出一个混合整数规划（MIP）模型。

## 模型构建

### 集合和下标

$i \in T$：建设基站的潜在地点的下标和集合。

$j \in R$：区域的下标和集合。

$G(T,R,E)$：定义在潜在基站建设地点集合 $T$、要覆盖的区域集合 $R$ 和边集合 $E$ 上的二部图，其中如果区域 $j \in R$ 可以被地点 $i \in T$ 的基站覆盖，则存在边 $(i,j) \in E$。

### 参数

$c_{i} \in \mathbb{R}^+$：在地点 $i$ 建设基站的成本。

$p_{j} \in \mathbb{N}$：区域 $j$ 的人口数量。

### 决策变量

$covered_{j} \in \{0, 1 \}$：如果区域 $j$ 被覆盖，则此变量等于 1；否则为 0。

$build_{i} \in \{0, 1 \}$：如果建设基站 $i$，则此变量等于 1；否则为 0。

### 目标函数

- **人口覆盖**：我们寻求最大化基站覆盖的总人口。

$$\text{Max} \quad Z = \sum_{j \in R} p_{j} \cdot covered_{j}$$

### 约束条件

- **覆盖约束**：对于每个区域 $j \in R$，确保必须选择至少一个覆盖该区域的基站。

$$\sum_{(i,j) \in E} build_{i} \geq covered_{j} \quad \forall j \in R$$

- **预算约束**：我们需要确保建设基站的总成本不超过分配的预算。

$$\sum_{i \in T} c_{i} \cdot build_{i} \leq \text{budget}$$

## Python 实现

### 示例
以下部分取自 [Gurobi 基站覆盖问题示例](https://github.com/Gurobi/modeling-examples/blob/master/cell_tower_coverage/cell_tower.ipynb)。

此示例考虑了6个基站和9个区域的二部图。下表说明了每个基站站点（行）覆盖哪些区域（列）。

| <i></i> | 区域 0 | 区域 1 | 区域 2 | 区域 3 | 区域 4 | 区域 5 | 区域 6 | 区域 7 | 区域 8 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- |  --- |
| 基站 0 | 1 | 1 | - | - | - | 1 | - | - |  - |
| 基站 1 | 1 | - | - | - | - | - | - | 1 |  1 |
| 基站 2 | - | - | 1 | 1 | 1 | - | 1 | - |  - |
| 基站 3 | - | - | 1 | - | - | 1 | 1 | - |  - |
| 基站 4 | 1 | - | 1 | - | - | - | 1 | 1 |  1 |
| 基站 5 | - | - | - | 1 | 1 | - | - | - |  1 |

各区域的人口数量如下表所示。

| <i></i> | 区域 0 | 区域 1 | 区域 2 | 区域 3 | 区域 4 | 区域 5 | 区域 6 | 区域 7 | 区域 8 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| 人口 | 523 | 690 | 420 | 1010 | 1200 | 850 | 400 | 1008 | 950 |

在各个地点建设基站的成本如下表所示。

| <i></i> | 成本（百万美元） |
| --- | --- |
| 基站 0 | 4.2 |
| 基站 1 | 6.1 |
| 基站 2 | 5.2 |
| 基站 3 | 5.5 |
| 基站 4 | 4.8 |
| 基站 5 | 9.2 | 

分配的预算为 $\$20,000,000$。

现在我们将展示如何通过 OptVerse 解决上述问题。

### 环境配置

要使用 OptVerse Python 模块，用户必须导入该包。

In [ ]:
from optvpy import *

### 数据准备

我们首先使用上述示例数据准备构建模型所需的数据：

In [ ]:
# 参数
budget = 20
regions = list(range(9))
population = {0: 523, 1: 690, 2: 420, 3: 1010, 4: 1200, 5: 850, 6: 400, 7: 1008, 8: 950}

sites = list(range(6))
coverage = {
    0: {0, 1, 5},
    1: {0, 7, 8},
    2: {2, 3, 4, 6},
    3: {2, 5, 6},
    4: {0, 2, 6, 7, 8},
    5: {3, 4, 8}
}
cost = {0: 4.2, 1: 6.1, 2: 5.2, 3: 5.5, 4: 4.8, 5: 9.2}

### 创建模型

要创建模型，我们必须首先实例化一个环境，并将其传递给模型对象的构造函数。

In [ ]:
env = OPTVEnv()
model = OPTVModel(env)

### 添加变量

接下来，我们向模型中添加 'build' 和 'is_covered' 决策变量。
我们通过变量添加接口设置目标函数的线性系数。

In [ ]:
# 决策变量
build = model.AddVars(sites, lb=0, ub=1, vtype=OPTV_BINARY, name="Build")
is_covered = model.AddVars(regions, lb=0, ub=1, vtype=OPTV_BINARY, name="Is_covered")

### 添加约束

现在我们向模型添加约束条件。

In [ ]:
# 约束条件
model.AddConstrs((sum(build[t] for t in sites if r in coverage[t]) >= is_covered[r]
              for r in regions), name="Build2cover")

model.AddConstr(sum(cost[s] * build[s] for s in sites) <= budget, name="budget")

### 设置目标函数

设置目标函数以最大化人口覆盖。

In [ ]:
# 目标函数
model.SetObjective(sum(population[r] * is_covered[r] for r in regions), OPTVSense.MAXIMIZE)

### 优化模型

最后，我们可以对问题进行优化求解。

In [ ]:
# 优化
model.Optimize()

## 结果分析

优化模型的结果显示了在 2000 万美元预算下可以覆盖的最大人口数量。
让我们看看实现该最优结果的解决方案。

该方案确定了在哪些地点建设基站。

In [ ]:
# 显示决策变量的最优值
for tower in sites:
    if build[tower].X > 0.5:
        print(f"\n 在地点 {tower} 建设基站。")

### 覆盖方案

该方案确定了哪些区域被所建基站覆盖。

In [ ]:
for region in regions:
    if is_covered[region].X > 0.5:
        print(f"\n 区域 {region}（人口 {population[region]}）被覆盖。")

### 性能指标

计算覆盖率和预算利用率指标。

In [ ]:
# 计算基站建设计划覆盖的人口百分比

total_population = sum(population[r] for r in regions)
covered_population = sum(population[r] * is_covered[r].X for r in regions)

coverage_percentage = round(100 * covered_population / total_population, 2)

print(f"\n 基站建设计划的人口覆盖率为：{coverage_percentage}%")
print(f" 覆盖人口总数：{int(covered_population)} / {total_population}")

## 结论

在本示例中，我们解决了基站覆盖问题，目标是在满足预算约束的情况下建设基站，为尽可能多的人口提供信号覆盖。
我们学习了如何将问题制定为混合整数规划模型。
此外，我们还学习了如何使用 OptVerse Python API 实现混合整数规划模型并求解。